In [1]:
import matplotlib.pyplot as plt
import numpy as np
import cv2
import glob
import random

In [2]:
img_list = sorted(glob.glob('./image2/*.jpg'))
mask_list = sorted(glob.glob('./mask2/*.jpg'))

print(len(img_list), len(mask_list))
print(img_list[0], mask_list[0])

7232 7232
./image2/001_000048.jpg ./mask2/001_000048.jpg


In [3]:
# Preprocessing
mask_cv = []
img_cv = []
for i in range(len(mask_list)):
    mask = cv2.imread(mask_list[i], 0)
    mask = cv2.resize(mask, (256,256))
    ret,mask = cv2.threshold(mask,127,255,cv2.THRESH_BINARY)
    mask_cv.append(mask)

# img 256x256
for i in range(len(img_list)):
    img = cv2.imread(img_list[i], 0)
    img = cv2.resize(img, (256, 256))
    img_cv.append(img)
    

In [4]:
indices = np.where(mask_cv[5] != [0])
y, x = indices[0], indices[1]
coordinates = np.array((x,y))
print(sum(x)/len(x), sum(y)/len(y))


170.20151133501258 134.5478589420655


In [ ]:
# Option 1 - BS
def randomCrop(img, mask):
    
    # Pixels that's inside Mask
    indices = np.where(mask != [0]) 
    pixel_num = len(indices[0]) # Number of white pixels
    
    if pixel_num > 100:
    
        while True:
        
            # Setting (0,128) is too tight
        
            x = random.randint(0, 80)
            y = random.randint(0, 80)
    
            mask = mask[x:x+128, y:y+128]
            img = img[x:x+128, y:y+128]
        
            indices_check = np.where(mask != [0])
    
            if len(indices_check[0] <= pixel_num*0.8): 
                #img_new.append(img)
                #mask_new.append(mask)
                break
            
            else:   # Some pixels aren't included
                continue
    
    else:
        
        x = random.randint(0, 80)
        y = random.randint(0, 80)
    
        mask = mask[x:x+128, y:y+128]
        img = img[x:x+128, y:y+128]
        
    
    return img, mask

In [ ]:
# Option 2 - Not bad
def randomCrop(img, mask, size):
    
    indices = np.where(mask != [0])
    y, x = indices[0], indices[1]
    a, b = (int(sum(x)/len(x)), int(sum(y)/len(y))) # Center of Gravity
    W = (max(x) - min(x))//2
    h = (max(y) - min(y))//2
    
    if int(a+W - size) < int(a-W):
        x = random.randint(max(0, int(a+W) - size), min(int(a-W), int(size)))
    else:
        x = random.randint(0, int(a+W) - size)
    
    if int(b+h - size) < int(b-h):
        y = random.randint(max(0, int(b+h) - size), min(int(b-h), int(size)))
    else:
        y = random.randint(0, int(b+h) - size)
    
    mask = mask[x:x+size, y:y+size]
    img = img[x:x+size, y:y+size]

    return img, mask

In [5]:
# Option 3 - Main
def randomCrop(img, mask, size):
    
    indices = np.where(mask != [0])
    y, x = indices[0], indices[1]
    a, b = (min(x), min(y)) # Lower Left Corner 
    W = (max(x) - min(x)) # Width  
    h = (max(y) - min(y)) # Height
    #print(a, b, W, h)
    
    if int(a+W - size) < int(a-W):
        x = random.randint(max(0, int(a+W) - size), min(int(a-W), int(size)))
    
    # Sometimes left is bigger 
    else:
        x = random.randint(0, int(a+W) - size)
    
    if int(b+h - size) < int(b-h):
        y = random.randint(max(0, int(b+h) - size), min(int(b-h), int(size)))
    else:
        y = random.randint(0, int(b+h) - size)
        
    #print(x,y)
    
    mask = mask[x:x+size, y:y+size]
    img = img[x:x+size, y:y+size]

    return img, mask


In [6]:
# Saving RandomCrop

for i in range(len(img_list)):
    
    img_path = img_list[i][9:19]
    mask_path = mask_list[i][8:18]
    
    for j in range(4):
        
        img, mask = randomCrop(img_cv[i], mask_cv[i], 128)
     
        cv2.imwrite('./crop_image/' + img_path + '_' + str(j+1) + '.jpg', img)
        cv2.imwrite('./crop_mask/' + mask_path + '_' + str(j+1) + '.jpg', mask)
        
